In [1]:
import pandas as pd

from time import sleep
from json import loads
import json
from kafka import KafkaConsumer, KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, FloatType

In [2]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("AAPL") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .config("spark.jars", "/path to driver/jars/sqljdbc42.jar") \
    .getOrCreate()

24/05/03 08:40:35 WARN Utils: Your hostname, Madhus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.206 instead (on interface en0)
24/05/03 08:40:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/nekki/.ivy2/cache
The jars for the packages stored in: /Users/nekki/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-03f5b523-993c-499c-807c-ba1b1d9453e2;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/nekki/anaconda3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 273ms :: artifacts dl 6ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#h

In [3]:

def read_from_kafka(topic):
    # Read data from Kafka
    raw_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092") \
        .option("subscribe", topic) \
        .option("startingOffsets", "earliest") \
        .load()

    # Cast key and value to string and filter out the undesired rows
    string_df = raw_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
                      .filter(~col("value").contains("Meta Data"))

    # Define the schema
    schema = StructType([
        StructField("1. open", StringType(), True),
        StructField("2. high", StringType(), True),
        StructField("3. low", StringType(), True),
        StructField("4. close", StringType(), True),
        StructField("5. volume", StringType(), True)
    ])

    # Parse the JSON data and select the desired columns
    parsed_df = string_df.withColumn("parsed_value", from_json("value", schema)) \
                         .select(
                             col("key").alias("_id"),
                             col("parsed_value.`1. open`").alias("open"),
                             col("parsed_value.`2. high`").alias("high"),
                             col("parsed_value.`3. low`").alias("low"),
                             col("parsed_value.`4. close`").alias("close"),
                             col("parsed_value.`5. volume`").alias("volume")
                         )

    return parsed_df



def process_stream(df, epoch_id):
    # Define JDBC URL
    jdbc_url = "connection_strings"
    # Define connection properties
    connection_properties = {
        "user": "*******",
        "password": "*********",
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    # Write the DataFrame to an Azure SQL database table
    df.write \
      .jdbc(url=jdbc_url, table="AAPL", mode="append", properties=connection_properties)



if __name__ == "__main__":
    # You can change the topic to any specific stock topic you are interested in
    topic = "stock_data_aapl"
    kafka_df = read_from_kafka(topic)
    
    # Use writeStream to set up the micro-batch processing
    query = kafka_df.writeStream \
        .outputMode("update") \
        .foreachBatch(process_stream) \
    .option("checkpointLocation", "/path to checkpoint directory/") \
    .start()
        
#         .start()

    query.awaitTermination()


24/05/03 08:40:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/05/03 08:40:42 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
24/05/03 08:40:49 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/Users/nekki/anaconda3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/nekki/anaconda3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    ans

KeyboardInterrupt: 